In [3]:
import jraph
import jax
import jax.numpy as jnp
import e3nn_jax as e3nn
import numpy as np


import sys
sys.path.append('..')


In [4]:
%load_ext autoreload

In [5]:
%autoreload 2
import train
import models
import input_pipeline_tf
import configs.haikumace as haikumace

In [22]:
# Mimic what we do in train.py.
config = haikumace.get_config()
rng = jax.random.PRNGKey(0)
rng, dataset_rng = jax.random.split(rng)

# Obtain graphs.
datasets = input_pipeline_tf.get_datasets(dataset_rng, config)
train_iter = datasets["train"].as_numpy_iterator()
init_graphs = next(train_iter)

# Set up dummy variables to obtain the structure.
rng, init_rng = jax.random.split(rng)
net = train.create_model(config, run_in_evaluation_mode=True)
params = jax.jit(net.init)(init_rng, init_graphs)

done focus_node_embeddings
target_species_probs Traced<ShapedArray(float32[1,5])>with<DynamicJaxprTrace(level=0/1)>
(1, 5) (1, 5)
Traced<ShapedArray(int32[1])>with<DynamicJaxprTrace(level=0/1)>


In [21]:
graphs = jraph.GraphsTuple(
    nodes=jnp.array([[1, 2, 3], [1, 2, 3], [4, 5, 6], [7, 8, 9]]),
    edges=jnp.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
    globals=jnp.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
    n_node=jnp.array([2, 1, 1]),
    n_edge=jnp.array([1, 1, 1]),
    senders=jnp.array([0, 0, 0]),
    receivers=jnp.array([0, 0, 0]),
)

In [40]:
num_graphs = 4
rng, *species_rngs = jax.random.split(jax.random.PRNGKey(0), num_graphs + 1)

target_species_probs = jnp.ones((num_graphs, models.NUM_ELEMENTS)) / models.NUM_ELEMENTS
target_species = jax.vmap(lambda key, p: jax.random.choice(
    key, models.NUM_ELEMENTS, p=p))((species_rngs,), target_species_probs)


ValueError: vmap got inconsistent sizes for array axes to be mapped:
  * most axes (4 of them) had size 2, e.g. axis 0 of argument key[0][0] of type uint32[2];
  * one axis had size 4: axis 0 of argument p of type float32[4,5]

In [43]:
e3nn.IrrepsArray("10x0e", jnp.arange(10))

10x0e [0 1 2 3 4 5 6 7 8 9]

In [47]:
import re
seed, start, end = [int(val) for val in re.findall(r'\d+', "fragments_seed01_from130944_to133920")]
start, end

(130944, 133920)

In [22]:
def segment_sample(probabilities, segment_ids, num_segments, rng):
    """Sample from a categorical distribution across each segment.
    Args:
        segment_ids: A 1D array of segment ids.
        probs: A 1D array of probabilities.
    Returns:
        A 1D array of samples.
    """
    def sample_for_segment(rng, i):
        return jax.random.choice(rng, node_indices, p=jnp.where(i == segment_ids, probabilities, 0.))
    
    node_indices = jnp.arange(len(segment_ids))
    rngs = jax.random.split(rng, num_segments)
    return jax.vmap(sample_for_segment)(rngs, jnp.arange(num_segments))

In [23]:
focus_logits = graphs.nodes.sum(axis=1)
probs = jraph.partition_softmax(focus_logits, graphs.n_node)
print(probs)
for seed in range(100):
    print(segment_sample(probs, jnp.asarray([0, 0, 1, 2]), 3, jax.random.PRNGKey(seed)))

[0.5 0.5 1.  1. ]
[0 2 3]
[1 2 3]
[0 2 3]
[0 2 3]
[0 2 3]
[1 2 3]
[0 2 3]
[1 2 3]
[1 2 3]
[1 2 3]
[1 2 3]
[0 2 3]
[0 2 3]
[1 2 3]
[1 2 3]
[0 2 3]
[1 2 3]
[0 2 3]
[0 2 3]
[1 2 3]
[0 2 3]
[1 2 3]
[1 2 3]
[1 2 3]
[0 2 3]
[0 2 3]
[0 2 3]
[0 2 3]
[1 2 3]
[0 2 3]
[1 2 3]
[1 2 3]
[0 2 3]
[0 2 3]
[1 2 3]
[1 2 3]
[0 2 3]
[0 2 3]
[0 2 3]
[1 2 3]
[0 2 3]
[1 2 3]
[1 2 3]
[0 2 3]
[0 2 3]
[1 2 3]
[1 2 3]
[0 2 3]
[1 2 3]
[1 2 3]
[0 2 3]
[1 2 3]
[1 2 3]
[1 2 3]
[0 2 3]
[0 2 3]
[0 2 3]
[0 2 3]
[1 2 3]
[1 2 3]
[1 2 3]
[0 2 3]
[1 2 3]
[0 2 3]
[1 2 3]
[1 2 3]
[1 2 3]
[1 2 3]
[0 2 3]
[0 2 3]
[0 2 3]
[0 2 3]
[1 2 3]
[0 2 3]
[0 2 3]
[1 2 3]
[0 2 3]
[0 2 3]
[1 2 3]
[1 2 3]
[0 2 3]
[1 2 3]
[0 2 3]
[1 2 3]
[1 2 3]
[1 2 3]
[1 2 3]
[0 2 3]
[0 2 3]
[1 2 3]
[1 2 3]
[0 2 3]
[1 2 3]
[1 2 3]
[1 2 3]
[1 2 3]
[1 2 3]
[0 2 3]
[1 2 3]
[0 2 3]


In [3]:
nodes = jnp.asarray([2, 3, 1, 4])
segment_max = e3nn.scatter_max(nodes, nel=graphs.n_node)
segment_max_expanded = e3nn.scatter_max(nodes, map_back=True, nel=graphs.n_node)
print(segment_max_expanded)
# segment_max_expanded = jnp.asarray([segment_max[0], segment_max[0], segment_max[1], segment_max[2]])

expected = 0 + jnp.log(1 + e3nn.scatter_sum(jnp.exp(nodes - 0), nel=graphs.n_node))
computed = segment_max + jnp.log(jnp.exp(-segment_max) + e3nn.scatter_sum(jnp.exp(nodes - segment_max_expanded), nel=graphs.n_node))

expected, computed

[3. 3. 1. 4.]


(Array([3.3490124, 1.3132616, 4.01815  ], dtype=float32),
 Array([3.3490121, 1.3132616, 4.01815  ], dtype=float32))

In [4]:
lmax = 3
irreps = e3nn.Irreps(e3nn.Irrep.iterator(lmax))
e3nn.IrrepsArray(irreps=irreps, array=jnp.ones((10, irreps.dim)))

1x0e+1x0o+1x1o+1x1e+1x2e+1x2o+1x3o+1x3e
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

In [5]:
e3nn.scatter_sum(data=graphs.nodes, nel=graphs.n_node)

Array([[2., 4., 6.],
       [4., 5., 6.],
       [7., 8., 9.]], dtype=float32, weak_type=True)